In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("Missing").getOrCreate()

In [6]:
#Read the data
df=spark.read.csv(r"C:\Users\faisal\Downloads\test1.csv",header=True,inferSchema=True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
feature_assember=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [12]:
output=feature_assember.transform(df)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [14]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [18]:
finalized_data = output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression

#train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

#train the linear Regression Model
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)

In [20]:
#coeficients
regressor.coefficients

DenseVector([-58.8235, 1666.6667])

In [21]:
#intercept
regressor.evaluate(test_data)

In [22]:
#prediction
pred_results=regressor.evaluate(test_data)


In [24]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18784.313725490192|
|          [30.0,8.0]| 25000|26764.705882352937|
+--------------------+------+------------------+



In [25]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1490.1960784313724, 2296039.984621296)